## Variables
Variables可以通过tf.Vaiable类来操纵。一个Variable代表一个可以通过执行操作而改变数值的tensor。不像其他tensor对象，tf.Variable可以存在于session.run内容以外。

从内部来说，一个tf.Variable存储着tensor的持久对象。指定操作允许你读写该tensor的数值。这些改动可以再多个tf.Session中可见，所以多个工作人员可以看到同样的tf.Variable值。

### 1.创建一个Variable
创建一个Variable最好的方式就是调用tf.get_variable函数，这个函数要求你指定Variable的名字，其他操作以及存储模型都会通过该名来获取特定的值。tf.get_variable同样也允许你重复使用一个过去使用的同样名字的variable，使重复定义模型的隐藏层更加容易。

示例代码如下 👇

In [2]:
import tensorflow as tf
my_variable = tf.get_variable("my_variable", [1, 2, 3])

In [3]:
my_variable

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>

该操作创建一个名为"my_variable"的三维tensor，shape为[1,2,3]。该Variable，将默认其数据类型为tf.float32并且将通过tf.glorot_uniform_initializer完成随机初始化。

当然你也可以自行设置数据类型以及初始化方法。

In [4]:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)

Tensorflow提供很多方便的初始化器，所以你也可以初始化一个tf.Variable具有tf.Tensor的值。

In [5]:
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))

不过需要注意的是，当用tensor作为初始化器时，不需指定Variable的shape，因为初始化器tensor的shape将被自动调用。

### 2.Variable 集合
因为或许互不相连的TensorFlow程序想要创建一些variable，此时有单一的方式获取Variable更加合适。出于这个原因，TensorFlow提供了collections，即命名一列的tensors或者其他对象。

默认来说，每个Variable都在以下两个集合中有自己的位置：
* tf.GraphKeys.GLOBAL_VARIABLES： 允许variable在多个设备间共享
* tf.GraphKeys.TRAINABLE_VARIABLES： 将被用于计算梯度的variables

如果你不想某个Variable被训练的话，将它加入tf.GraphKeys.LOCAL_VARIABLES集合，例如：

In [6]:
my_local = tf.get_variable("my_local", shape=(),
collections=[tf.GraphKeys.LOCAL_VARIABLES])

或者说，你也可以在tf.get_variable方法中指定trainable参数为False。

In [7]:
my_non_trainable = tf.get_variable("my_non_trainable",
                                   shape=(),
                                   trainable=False)

你也可以定义你自己的集合。任何string类型都是有效的集合名称，并且并不需要清晰地定义一个集合。为了在创建Variable后将其加入指定集合，可以调用tf.add_to_collection方法。

例如，以下代码，将my_local的Variable加入到my_collection_name的集合中：

In [8]:
tf.add_to_collection("my_collection_name", my_local)

以及，查看某个collection中所有的variable

In [9]:
tf.get_collection("my_collection_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

### 3. 硬件设置
如同其他TensorFlow操作一样，你可以放置variables到指定的设备中，例如：

(温馨提示：没有gpu的不用按shift+enter了

In [ ]:
with tf.device("/device:GPU:1"):
    v = tf.get_variable("v", [1])

对于Variable来说，在分布式设置中，放置在正确的设备尤其重要。Tensorflow给出了一些自动化设置：

In [ ]:
cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v = tf.get_variable("v", shape=[20, 20])  # this variable is placed
                                            # in the parameter server
                                            # by the replica_device_setter

### 4.初始化Variables
在你使用一个Variable之前，必须先经过初始化，如果你正在用tensorflow的底层API进行编程的话（就是说，你在自行创建Graph和Session），你必须先初始化这些变量。大多数高层API例如tf.contrib.slim，tf.estimator.Estimator以及Keras在你训练变量之前，自动的初始化变量。

如果想一次性的初始化所有Variable，可以调用tf.global_variables_initializer()，该方法初始化所有在tf.GraphKeys.GLOBAL_VARIABLES中的变量。

示例如下👇

In [10]:
session = tf.Session()
session.run(tf.global_variables_initializer())
# Now all variables are initialized.

如果你确实需要手动初始化的话，可以调用以下方法：

In [11]:
session.run(my_variable.initializer)

以及，如果需要查看还有哪些变量尚未被初始化的话：

In [12]:
print(session.run(tf.report_uninitialized_variables()))

[b'my_local']


注意，默认来说，tf.global_variables_initializer并不指定变量初始化的顺序，因此，如果一个变量初始化的值依赖于另一个变量，那很有可能会出现报错情况。所以，当不是所有变量都已经初始化完成时，如果你想使用另外一个变量，最好调用variable.initialized_value()方法：

In [13]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)

### 5. 使用Variable
如果想使用Graph中某个Variable的值，只需要像使用其他tensor一样：

In [ ]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1
#w = v + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.

如果你想给某个Variable部署特定值的话，可以用assign方法：

In [ ]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
tf.global_variables_initializer().run()
sess.run(assignment)  # or assignment.op.run(), or assignment.eval()

大多数Tensorflow优化器针对每个优化算法有指定有效的优化方法，具体看<a href="https://www.tensorflow.org/api_docs/python/tf/train/Optimizer?hl=zh-cn">tf.train.Optimizer </a>了解如何使用。

因为变量间相互作用，所以有时知道某个Variable在使用哪个版本的值是很有用的。如果想在有些程序已经执行后，强行重读某个Variable的值，你可以使用tf.Variable.read_value方法，例如：

In [ ]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
    w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.

### 6.分享变量
Tensorflow支持两种分享变量的方法：
* 显示传递tf.Variable对象
* 用tf.variable_scope包装tf.Variable后非显示传递

虽然使用显示传递Variable非常清晰，但有时使用非显示传递更为方便。tf.layer,tf.metrics以及一些其他lib都使用相关方法。

Variable的scope特性允许你控制Variable的重复使用，特别是调用函数时非显示的创建并使用Variable。它们同样允许你以一种垂直且容易理解的方式来给你的Variable命名。

例如，我们来创建一个卷积层：

In [18]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

该函数将创建的weights和biases简称为“weights”，“biases”。尽管这样很清晰，然而在实际模型中，我们需要很多类似的卷积层，重复调用该函数并不起作用。

会出现以下的报错信息：

In [19]:
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.

ValueError: Variable weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-18-5edf5d2e6e72>", line 4, in conv_relu
    initializer=tf.random_normal_initializer())
  File "<ipython-input-19-917762734a05>", line 3, in <module>
    x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


由于以上目的并不清楚：是创建新变量还是重复使用已存在的变量？
Tensorflow执行失败，在不同的scope中调用conv_relu，意味着我们想要创建新变量。

In [20]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

如果你确实想要分享变量的话，你有两种选择，意识你可以创建统一名称的scope，并把参数reuse设置为True：

In [21]:
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model", reuse=True):
    output2 = my_image_filter(input2)


或者你也可以调用scope.reuse_variables()来激活reuse操作：

In [23]:
with tf.variable_scope("model1") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)

因为zai